In [1]:
#生成矩形数据的函数
def get_rectangle():
    import random

    #随机得到矩形的宽和高,值域0-1之间的小数
    width = random.random()
    height = random.random()

    #判断这是否是一个"胖"的矩形
    fat = int(width >= height)

    return width, height, fat


get_rectangle()

(0.7363799113739945, 0.04611079149918318, 1)

In [2]:
import torch


#定义数据集
class Dataset(torch.utils.data.Dataset):

    #正常应该在这里执行数据的加载,处理等工作
    def __init__(self):
        pass

    #定义数据的条数
    def __len__(self):
        return 500

    #根据序号i,获取一条数据
    def __getitem__(self, i):
        #获取一个矩形的数据
        width, height, fat = get_rectangle()

        #定义宽高为x,定义是否胖为y
        x = torch.FloatTensor([width, height])
        y = fat

        return x, y


dataset = Dataset()

len(dataset), dataset[0]

(500, (tensor([0.2017, 0.8151]), 0))

In [3]:
#数据集加载器,每8条数据为一个批次,打乱顺序,不足8条时丢弃尾数
loader = torch.utils.data.DataLoader(dataset=dataset,
                                     batch_size=8,
                                     shuffle=True,
                                     drop_last=True)

len(loader), next(iter(loader))

(62,
 [tensor([[0.4344, 0.8545],
          [0.3336, 0.6493],
          [0.7499, 0.2309],
          [0.4290, 0.6426],
          [0.5168, 0.8115],
          [0.7426, 0.4381],
          [0.5285, 0.1795],
          [0.5070, 0.6046]]),
  tensor([0, 0, 1, 0, 0, 1, 1, 0])])

In [4]:
#全连接神经网络
class Model(torch.nn.Module):

    #模型初始化部分
    def __init__(self):
        super().__init__()

        #定义神经网络结构
        self.fc = torch.nn.Sequential(
            torch.nn.Linear(in_features=2, out_features=32),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=32, out_features=32),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=32, out_features=2),
            torch.nn.Softmax(dim=1),
        )

    #定义神经网络计算过程
    def forward(self, x):
        return self.fc(x)


model = Model()

model(torch.randn(8, 2)).shape

torch.Size([8, 2])

In [5]:
#训练
def train():
    #优化器,根据梯度调整模型参数
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    
    #计算loss的函数
    loss_fun = torch.nn.CrossEntropyLoss()
    
    #让model进入train模式,开启dropout等功能
    model.train()

    #全量数据遍历100轮
    for epoch in range(100):
        
        #按批次遍历loader中的数据
        for i, (x, y) in enumerate(loader):
            
            #模型计算
            out = model(x)
            
            #根据计算结果和y的差,计算loss,在计算结果完全正确的情况下,loss为0
            loss = loss_fun(out, y)

            #根据loss计算模型的梯度
            loss.backward()
            
            #根据梯度调整模型的参数
            optimizer.step()
            
            #梯度归零,准备下一轮的计算
            optimizer.zero_grad()

        if epoch % 20 == 0:
            #计算正确率
            acc = (out.argmax(dim=1) == y).sum().item() / len(y)
            print(epoch, loss.item(), acc)
            
    #保存模型到磁盘
    torch.save(model, 'model/3.model')


train()

0 0.6785416603088379 0.625
20 0.47304677963256836 1.0
40 0.367154598236084 1.0
60 0.4003196954727173 1.0
80 0.37078338861465454 1.0


In [6]:
#测试
#注释的表明不计算模型梯度,节省计算资源
@torch.no_grad()
def test():
    
    #从磁盘加载模型
    model = torch.load('model/3.model')

    #模型进入测试模式,关闭dropout等功能
    model.eval()

    #获取一批数据
    x, y = next(iter(loader))

    #模型计算结果
    out = model(x).argmax(dim=1)

    print(out, y)
    print(out == y)


test()

tensor([1, 1, 1, 0, 0, 0, 0, 0]) tensor([1, 1, 1, 0, 0, 0, 0, 0])
tensor([True, True, True, True, True, True, True, True])
